## 一.导入包

In [ ]:
import os
import ast
import argparse
import mindspore.nn as nn
from mindspore import context
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train import Model
from mindspore.nn.metrics import Accuracy
from src.dataset import create_dataset
from src.config import mnist_cfg as cfg
from src.lenet import LeNet5

## 二.设备设置

In [ ]:
parser = argparse.ArgumentParser(description='MindSpore Lenet Example')
parser.add_argument('--device_target', type=str, default="CPU", choices=['Ascend', 'GPU', 'CPU'],
                        help='device where the code will be implemented (default: Ascend)')
parser.add_argument('--data_path', type=str, default="./MNIST_Data",
                        help='path where the dataset is saved')
                        #设置验证问文件
parser.add_argument('--ckpt_path', type=str, default="ckpt/checkpoint_lenet-10_1875.ckpt", help='if mode is test, must provide\
                        path where the trained ckpt file')
parser.add_argument('--dataset_sink_mode', type=ast.literal_eval,
                        default=False, help='dataset_sink_mode is False or True')

args = parser.parse_known_args()[0]

context.set_context(mode=context.GRAPH_MODE, device_target=args.device_target)

## 三.预备工作

In [ ]:
network = LeNet5(cfg.num_classes)
#设定loss函数
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
repeat_size = cfg.epoch_size
#设定优化器
net_opt = nn.Momentum(network.trainable_params(), cfg.lr, cfg.momentum)
#编译形成模型
model = Model(network, net_loss, net_opt, metrics={"Accuracy": Accuracy()})

## 四.开始测试

In [ ]:
param_dict = load_checkpoint(args.ckpt_path)
load_param_into_net(network, param_dict)
ds_eval = create_dataset(os.path.join(args.data_path, "test"),
                             cfg.batch_size, # type: ignore
                             1)
acc = model.eval(ds_eval, dataset_sink_mode=args.dataset_sink_mode)
print("============== {} ==============".format(acc))